In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import applications
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

batch_size = 16
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20

Using TensorFlow backend.
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

# Parse do dataset e separação entre conjuntos de teste e treinamento

In [2]:
# auxiliary function
def depth_two_uint8_to_float(top_bits, bottom_bits):
    """ Converts a RGB-coded depth into float valued depth. """
    depth_map = (top_bits * 2**8 + bottom_bits).astype('float32')
    depth_map /= float(2**16 - 1)
    depth_map *= 5.0
    return depth_map

# load annotations of this set
with open(os.path.join(set, 'anno_%s.pickle' % set), 'rb') as fi:
    anno_all = pickle.load(fi)

# iterate samples of the set
for sample_id, anno in anno_all.items():
    # load data
    image = cv2.imread(os.path.join(set, 'color', '%.5d.png' % sample_id))
    mask = cv2.imread(os.path.join(set, 'mask', '%.5d.png' % sample_id))
    mask = cv2.normalize(mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    X.append(image)
    y.append(mask)

    cv2.imshow("image", image)
    cv2.imshow("mask", mask)
    cv2.waitKey(0)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)


In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
start_time = time.time()

model2.fit(x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(x_test, y_test),
      shuffle=True)
print("Fitting duration: " + str(time.time() - start_time))

acc = model2.evaluate(x_test, y_test)
print("Test accuracy:" + str(acc[1]))
print("Loss:" + str(acc[0]))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 366s 7ms/step - loss: 2.3828 - accuracy: 0.2712 - val_loss: 1.7033 - val_accuracy: 0.4109
Epoch 2/5
50000/50000 [==============================] - 366s 7ms/step - loss: 1.7298 - accuracy: 0.4191 - val_loss: 1.5503 - val_accuracy: 0.4905
Epoch 3/5
50000/50000 [==============================] - 373s 7ms/step - loss: 1.4785 - accuracy: 0.4972 - val_loss: 1.6820 - val_accuracy: 0.4585
Epoch 4/5
50000/50000 [==============================] - 384s 8ms/step - loss: 1.3072 - accuracy: 0.5522 - val_loss: 1.2296 - val_accuracy: 0.5829
Epoch 5/5
50000/50000 [==============================] - 378s 8ms/step - loss: 1.1807 - accuracy: 0.5991 - val_loss: 1.2092 - val_accuracy: 0.6138
Fitting duration: 1876.0506908893585
10000/10000 [==============================] - 10s 963us/step
Test accuracy:0.6137999892234802
Loss:1.2092481250762939


## Mesmo com um tempo de treinamento muito maior e uma estrutura mais complexa, a rede RESNET obteve resultado muito proximo á rede neural criada anteriormente, isso pode ser explicado por:
### A rede RESNET necessita de mais tempo de treinamento, devido a quantidade muito maior de camadas.
### Possivelmente essa acurácia melhoraria com mais épocas em seu treinamento.

In [ ]:
model2.metrics_names

NameError: name 'model2' is not defined